문제 출처 : https://school.programmers.co.kr/learn/courses/30/lessons/42893

In [1]:
from typing import List, Iterator, Union
import re

def get_url(page: str, url_pattern: re.Pattern) -> str:
    """페이지로부터 URL을 반환하는 함수

    page : 페이지
    url_pattern : URL을 추출하는 정규표현식 패턴
    """
    return url_pattern.search(page).group("url")


def get_base_score(word: str, page: str, body_pattern: re.Pattern, href_split_pattern: re.Pattern, word_pattern: re.Pattern) -> int:
    """페이지에 있는 단어의 갯수를 반환하는 함수

    word : 검색하는 단어
    page : 페이지
    body_pattern : 페이지에서 body태그를 추출하는 정규표현식 패턴
    href_split_pattern : 페이지에서 외부 링크들을 기준으로 나누는 정규표현식 패턴
    word_pattern : 페이지에서 단어들을 추출하는 패턴
    """
    # body태그를 추출
    body = body_pattern.search(page).group().strip()
    # 외부 링크들을 기준으로 나누기
    text_list = href_split_pattern.split(body)
    # 단어 추출하기
    word_list = []
    for text in text_list:
        words = word_pattern.findall(text)
        if words:
            word_list.extend(words)

    return word_list.count(word)


def get_href(page: str, href_pattern: re.Pattern) -> Iterator[Union[int, str]]:
    """페이지로부터 외부 링크의 수 및 외부 링크들을 반환하는 함수

    page : 페이지
    href_pattern : 페이지에서 외부 링크들을 추출하는 정규표현식 패턴
    """
    href_list = href_pattern.findall(page)
    yield len(href_list)
    yield from href_list


def solution(word: str, pages: List[str]) -> int:
    word = word.lower()
    pages = [page.lower() for page in pages]
    url_pattern = re.compile(r'<meta\s*property=\"og:url\"\s*content=\"(?P<url>.+?)\"/>') # URL을 추출하는 정규표현식 패턴
    body_pattern = re.compile(r'(?<=<body>).+(?=</body>)', re.DOTALL) # body태그를 추출하는 정규표현식 패턴
    href_split_pattern = re.compile(r'<a href=\".+?</a>') # 외부 링크들을 기준으로 나누는 정규표현식 패턴
    href_pattern = re.compile(r'(?<=<a href=\").+?(?=\">)') # 외부 링크들을 추출하는 정규표현식 패턴
    word_pattern = re.compile(r'[a-z]+') # 단어들을 추출하는 패턴
    page_dict = {get_url(page, url_pattern): {"idx": idx, "base_score": 0, "outside_count": 0, "href_list": [], "link_score": 0} for idx, page in enumerate(pages)}
    max_score = 0
    answer = 0

    # 각 페이지의 기본점수 및 외부 링크 수, 외부 링크들 계산
    for page in pages:
        url = get_url(page, url_pattern)
        href_list = get_href(page, href_pattern)
        page_dict[url]["base_score"] += get_base_score(word, page, body_pattern, href_split_pattern, word_pattern)
        page_dict[url]["outside_count"] += next(href_list)
        for href in href_list:
            # 페이지 목록에 있는 외부 링크들만 추가
            if href in page_dict:
                page_dict[href]["href_list"].append(url)

    # 각 페이지의 링크점수 및 매칭점수 계산
    for url in page_dict:
        page_dict[url]["link_score"] += sum(page_dict[linked]["base_score"] / page_dict[linked]["outside_count"] for linked in page_dict[url]["href_list"])
        page_dict[url]["match_score"] = page_dict[url]["base_score"] + page_dict[url]["link_score"]
        if max_score < page_dict[url]["match_score"]:
            max_score = page_dict[url]["match_score"]
            answer = page_dict[url]["idx"]

    return answer

# TestCase
print(solution("blind", ["<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://a.com\"/>\n</head>  \n<body>\nBlind Lorem Blind ipsum dolor Blind test sit amet, consectetur adipiscing elit. \n<a href=\"https://b.com\"> Link to b </a>\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://b.com\"/>\n</head>  \n<body>\nSuspendisse potenti. Vivamus venenatis tellus non turpis bibendum, \n<a href=\"https://a.com\"> Link to a </a>\nblind sed congue urna varius. Suspendisse feugiat nisl ligula, quis malesuada felis hendrerit ut.\n<a href=\"https://c.com\"> Link to c </a>\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://c.com\"/>\n</head>  \n<body>\nUt condimentum urna at felis sodales rutrum. Sed dapibus cursus diam, non interdum nulla tempor nec. Phasellus rutrum enim at orci consectetu blind\n<a href=\"https://a.com\"> Link to a </a>\n</body>\n</html>"]) == 0)
print(solution("Muzi", ["<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://careers.kakao.com/interview/list\"/>\n</head>  \n<body>\n<a href=\"https://programmers.co.kr/learn/courses/4673\"></a>#!MuziMuzi!)jayg07con&&\n\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://www.kakaocorp.com\"/>\n</head>  \n<body>\ncon%\tmuzI92apeach&2<a href=\"https://hashcode.co.kr/tos\"></a>\n\n\t^\n</body>\n</html>"]) == 1)

True
True
